In [35]:
!pip -q install folium

In [36]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import branca.colormap as cm
from IPython.display import HTML
from folium.plugins import MousePosition 
import warnings
from folium import Popup
warnings.filterwarnings('ignore')

In [37]:
data_collar = pd.read_csv("datasets/Collar.csv")
data_litologi = pd.read_csv("datasets/Litologi.csv")
data_assay = pd.read_csv("datasets/Assay.csv")

In [38]:
data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_assay['Hole_id'] = data_assay['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_litologi['hole_id'] = data_litologi['hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [39]:
# data_collar['X'] = data_collar['X'].astype(float)
# data_collar['Y'] = data_collar['Y'].astype(float)
# # data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [40]:
new_data = pd.concat([
    data_assay[['Hole_id']],
    data_litologi[['dept_from', 'Dept_to', 'Litho']].rename(columns={
        'dept_from': 'Depth_from',
        'Dept_to': 'Depth_to'
    }),
    data_assay[[' Ni']].rename(columns={' Ni': 'Ni'})
], axis=1)

# save csv
new_data.to_csv('datasets/new_data.csv', index=False)



In [41]:
sap_data = new_data[new_data['Litho'] == 'SAP'].copy()
sap_data['Tebal_Per_Sampel'] = sap_data['Depth_to'] - sap_data['Depth_from']
sap_data.to_csv('datasets/sap_data.csv', index=False)

In [42]:
sap_data['Weighted_Ni'] = sap_data['Ni'] * sap_data['Tebal_Per_Sampel']

In [43]:
aggregated_data = sap_data.groupby('Hole_id').agg(
    Depth_from = ('Depth_from', 'min'),
    Depth_to = ('Depth_to', 'max'),
    Total_SAP = ('Tebal_Per_Sampel', 'sum'),
    Min_Ni = ('Ni', 'min'),
    Max_Ni = ('Ni', 'max')
).reset_index()

In [44]:
weighted_sum = sap_data.groupby('Hole_id')['Weighted_Ni'].sum().reset_index(name='Weighted_Sum_Ni')

In [45]:
aggregated_data = pd.merge(aggregated_data, weighted_sum, on='Hole_id')

In [46]:
aggregated_data['Avg_Ni'] = aggregated_data['Weighted_Sum_Ni'] / aggregated_data['Total_SAP']

In [47]:
aggregated_data.drop(columns=['Weighted_Sum_Ni'], inplace=True)
aggregated_data.to_csv('datasets/aggregated_data.csv', index=False) 

In [48]:
# # Filter data untuk litologi SAP
# sap_data = new_data[new_data['Litho'] == 'SAP'].copy()

# # Hitung ketebalan per baris
# sap_data['Thickness'] = sap_data['Depth_to'] - sap_data['Depth_from']

# # Hitung total ketebalan, min, dan max Ni per Hole_id
# summary = sap_data.groupby('Hole_id').agg(
#     Depth_from=('Depth_from', 'min'),
#     Depth_to=('Depth_to', 'max'),
#     Total_SAP=('Thickness', 'sum'),
#     Min_Ni=('Ni', 'min'),
#     Max_Ni=('Ni', 'max')
# ).reset_index()

# # Hitung total tertimbang Ni per Hole_id
# sap_data['Weighted_Ni'] = sap_data['Ni'] * sap_data['Thickness']
# weighted_sum = sap_data.groupby('Hole_id')['Weighted_Ni'].sum().reset_index(name='Weighted_Sum_Ni')

# # Gabungkan dengan summary
# aggregated_data = pd.merge(summary, weighted_sum, on='Hole_id')

# # Hitung rata-rata tertimbang
# aggregated_data['Avg_Ni'] = aggregated_data['Weighted_Sum_Ni'] / aggregated_data['Total_SAP']

# # Hapus kolom antara jika tidak ingin disimpan
# aggregated_data.drop(columns=['Weighted_Sum_Ni'], inplace=True)

# # Simpan ke file
# aggregated_data.to_csv('datasets/aggregated_data.csv', index=False)

# # Tampilkan hasil
# aggregated_data.head()


In [49]:
# Combine all relevant columns for plotting data
plot_data = pd.concat([
    aggregated_data[['Hole_id']],
    data_collar[['X', 'Y']],
    aggregated_data[['Depth_from', 'Depth_to', 'Total_SAP', 'Avg_Ni', 'Min_Ni', 'Max_Ni']]
], axis= 1)

# save to .csv file
plot_data.to_csv('datasets/plot_data.csv', index=False)
plot_data.head()


,Hole_id,X,Y,Depth_from,Depth_to,Total_SAP,Avg_Ni,Min_Ni,Max_Ni
0,1.0,-2.507508,121.513665,11.0,16.0,5.0,1.680500,1.03,2.08
1,2.0,-2.507511,121.512784,9.0,24.0,15.0,2.312200,1.38,3.07
2,3.0,-2.507511,121.511902,4.4,9.0,4.6,1.915652,0.83,2.53
3,4.0,-2.507515,121.510981,8.0,13.0,5.0,1.614000,1.25,2.17
4,5.0,-2.507488,121.510041,5.0,10.0,5.0,1.724000,1.50,2.13


In [50]:
# plt.figure(figsize=(15, 8))
# plt.scatter(data_collar['Y'], data_collar['X'], c='blue', alpha=0.5)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.title('Peta titik dari data collar')
# plt.grid(True)
# plt.show()

In [51]:
# Define bounds for map view
min_lat, max_lat = plot_data['X'].min(), plot_data['X'].max()
min_lon, max_lon = plot_data['Y'].min(), plot_data['Y'].max()

# Define value ranges for color mapping
Min_Avg_Ni = plot_data['Avg_Ni'].min()
Max_Avg_Ni = plot_data['Avg_Ni'].max()

# Create base map with Esri
map = folium.Map(
    zoom_start=17,
    min_zoom=10,
    control_scale= True,
    font_size='1.5rem',
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri — Source: Esri'
)

# Fitting map to data bounds
map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Create color scale
colormap = cm.linear.YlOrRd_09.scale(Min_Avg_Ni, Max_Avg_Ni)
colormap.caption = ' Avg. Nickel Value Scale'
colormap.add_to(map)

# Custom CSS for colormap legend background
custom_css = """
<style>
.legend {
    background-color: rgba(255, 255, 255, 0.85);
    padding: 8px 12px;
    border-radius: 6px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.3);
    font-size: 13px;
}
</style>
"""
map.get_root().header.add_child(folium.Element(custom_css))

# Add mouse position to bottom right of map
mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)

# Add circle markers for each row
for _, row in plot_data.iterrows():
    # color for the circle
    color = colormap(row['Avg_Ni'])
    # radius size of each circle
    radius = 2 + row['Total_SAP'] * 0.7

    folium.CircleMarker(
        location=[row['X'], row['Y']],  
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        tooltip=f"<b>BH:</b> {row['Hole_id']:.0f}<br>",
        popup=Popup(
        html=(
            f"<b>BH:</b> {row['Hole_id']:.0f}<br>"
            f"<b>Lat:</b> {row['X']:.5f}<br>"
            f"<b>Lon:</b> {row['Y']:.5f}<br>"
            f"<b>SAP:</b> {row['Total_SAP']:.2f} meter<br>"
            f"<b>Avg. Ni:</b> {row['Avg_Ni']:.2f}<br>"
            f"<b>Min. Ni:</b> {row['Min_Ni']:.2f}<br>"
            f"<b>Max. Ni:</b> {row['Max_Ni']:.2f}<br>"
        ),
        max_width=200  
    )
    ).add_to(map)

# Displaying the map
# The map will fill the entire output cell, so it's customized through css
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))
# map # uncomment this to use show full size of the map

In [57]:
# Buat Grid
gridx = np.linspace(plot_data['X'].min(), plot_data['X'].max(), 100)
gridy = np.linspace(plot_data['Y'].min(), plot_data['Y'].max(), 100)
xi, yi = np.meshgrid(gridx, gridy)

In [70]:
from pykrige.ok import OrdinaryKriging

# ambil nilai x, y, z (avg_ni)
x = plot_data['X'].values
y = plot_data['Y'].values
z = plot_data['Avg_Ni'].values

# Buat objek kriging
OK = OrdinaryKriging(
    x, y, z,
    variogram_model= "gaussian", # linear, gaussian, spherical, exponential
    verbose= False,
    enable_plotting= False
)

# Interpolasi ke grid
zi, ss = OK.execute('grid', gridx, gridy)

In [73]:
"""""
Buat map interpolasi avg_ni pakai plotly
Kenapa plotly? Karena plotly support peta contour, sedangkan folium tidak
"""
import plotly.graph_objects as go 

fig = go.Figure()

# buat contour / heatmap
fig.add_trace(go.Contour(
    z= zi,
    x= gridx,
    y= gridy,
    colorscale= 'Viridis',
    contours= dict(showlabels= True),
    colorbar= dict(title= 'Avg Ni Scale')
))

# Buat titik borehole
fig.add_trace(go.Scatter(
    x= plot_data['X'],
    y= plot_data['Y'],
    mode= 'markers+text',
    text= [f"{v:.0f}" for v in plot_data['Hole_id']],
    textposition= 'top center', 
    marker= dict(color= 'red', size= 8, line= dict(color= 'black', width= 1)),
    name= 'Borehole'
))

fig.update_layout(
    title= 'Interpolated Borehole Map',
    xaxis_title= 'X',
    yaxis_title= 'Y', 
    width= 800,
    height= 600
)

fig.show()